In [1]:
import torch
from torch._C import unify_type_list
import torch.nn as nn                     # neural networks
import torch.autograd as autograd         # computation graph
import numpy as np
from pyDOE import lhs

In [32]:
class PINNBTPDENN(nn.Module):
    def __init__(self,layerslist):
        super(PINNBTPDENN, self).__init__()
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')

        self.diffusivity = 2e-3
        self.bvalue = 500

        # self.flatten = nn.Flatten()
        modules = []
        for i in range(0,len(layerslist)-1):
            nnlayer = nn.Linear(layerslist[i], layerslist[i+1]).to(torch.cfloat)
            nn.init.xavier_normal_(nnlayer.weight.data)
            nn.init.zeros_(nnlayer.bias.data)
            modules.append(nnlayer)
            if i != len(layerslist)-2:
                modules.append(nn.Tanh())

        self.layers = nn.Sequential(*modules)
        # alternative choice is nn.init.normal_ or nn.init.kaiming_uniform_ kaiming_normal_

        self.iter = 0


    def forward(self, X):
        # X is stack of x and t
        if torch.is_tensor(X) != True:         
            X = torch.from_numpy(X)

        H = self.layers(X.cfloat())
        return H
    
    def compute_u_1order(self, x):
        self.u_x = torch.autograd.functional.jacobian(self, x, create_graph=True)
        self.u_x = torch.squeeze(self.u_x)
        return self.u_x
    
    def compute_u_2order(self, x):
        self.u_xx = torch.autograd.functional.jacobian(self.compute_u_1order, x)
        self.u_xx = torch.squeeze(self.u_xx)
        return self.u_xx
    
    def loss_PDE(self, x,y):
                
        # x_1_f = x_to_train_f[:,[0]]
        # x_2_f = x_to_train_f[:,[1]]
                        
        g = x.clone().cfloat()
                        
        g.requires_grad = True
        
        u = self.forward(g)
                
        u_1order = autograd.grad(u,g,torch.ones_like(u), retain_graph=True, create_graph=True)[0] 
        u_2order = autograd.grad(u_1order[:,[0]],g,torch.ones_like(u_1order[:,[0]]), create_graph=True)
        u_2order = autograd.grad(u_1order[:,[1]],g,torch.ones_like(u_1order[:,[1]]), create_graph=True)
#         u_2order = [0,0]

#         u_1order = self.compute_u_1order(g)
#         u_2order = self.compute_u_2order(g)
        
                
        return u_1order, u_2order


In [43]:
nnlayers = [4, 20, 20, 20, 20, 20, 20, 20, 20, 1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PINNBTPDENN(nnlayers).to(device)
radius = 1
demiheight = 50
N_f_train = 5
lb = np.array([-radius,-radius,-demiheight,0])
ub = np.array([radius,radius,demiheight,15000])
Input_f_train = lb + (ub-lb)*lhs(4,N_f_train) 
print(Input_f_train[:,[0]])
Input_f_train = torch.from_numpy(Input_f_train).float().to(device)
print(torch.gt(Input_f_train[:,[3]], 500).int() - torch.gt(Input_f_train[:,[3]], 500).int() )
zero_f_train = torch.zeros(Input_f_train.shape[0],1).to(device)

[[ 0.72928591]
 [-0.66543458]
 [-0.36342795]
 [ 0.56790124]
 [-0.02045112]]
tensor([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1]], dtype=torch.int32)


In [34]:
u_1order, u_2order = model.loss_PDE(Input_f_train,zero_f_train)
print(Input_f_train.size())
print(u_1order)
print(u_1order[:,[0]])
print(u_2order)

torch.Size([5, 4])
tensor([[ 0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j],
        [ 1.4190e-35-8.1579e-36j, -4.4163e-35+6.0375e-35j,
         -8.8659e-35-1.1268e-35j,  1.8195e-35-1.1584e-34j],
        [ 0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j]],
       grad_fn=<MmBackward>)
tensor([[0.0000e+00+0.0000e+00j],
        [1.4190e-35-8.1579e-36j],
        [0.0000e+00+0.0000e+00j],
        [0.0000e+00+0.0000e+00j],
        [0.0000e+00+0.0000e+00j]], grad_fn=<IndexBackward>)
(tensor([[ 0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j],
        [ 7.2156e-37+6.1466e-36j, -1.440

In [36]:
a,b,c = ([1,2],[3,4],[5,6])
print(a,b,c)

[1, 2] [3, 4] [5, 6]


In [ ]:
np.sqrt(-1)